<a href="https://colab.research.google.com/github/alexrofail/Loss-Optimizers-Training-Loops/blob/main/callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn.functional as F
import gzip
import pickle
from torch import tensor
from torch import nn

In [2]:
import pickle, gzip, math, torch, matplotlib as mpl

In [3]:
def accuracy(out, yb):
  return (torch.argmax(out, dim=1)==yb).float().mean()

In [4]:
from torch import optim
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [5]:
class Dataset():
  def __init__(self,x,y):
    self.x ,self.y = x,y
  
  def __len__(self):
    return len(self.x)

  def __getitem__(self,i):
    return self.x[i], self.y[i]

In [6]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
  return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
          DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [11]:
def get_data():
    with gzip.open('mnist.pkl.gz') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [12]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,64
c = y_train.max().item()+1
loss_func = F.cross_entropy

In [ ]:
#TODO: very important, go back and fix the exports thing 
#TODO: cont... so that you don't have to c/p code above to maintain library

In [15]:
#export
class Databunch():
  def __init__(self, train_dl, valid_dl, c=None):
    self.train_dl, self.valid_dl, self.c = train_dl, valid_dl, c

  @property
  def train_ds(self): return self.train_dl.dataset

  @property
  def valid_ds(self): return self.valid_dl.dataset

In [16]:
data = Databunch(*get_dls(train_ds, valid_ds, bs), c)

In [20]:
#export
def get_model (data, lr=0.5, nh=50):
  m = data.train_ds.x.shape[1]
  model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, data.c))
  return model, optim.SGD(model.parameters(), lr=lr)


class Learner():
  def __init__(self, model, opt, loss_func, data):
    self.model, self.opt, self.loss_func, self.data = model, opt, loss_func, data

In [21]:
learn = Learner(*get_model(data), loss_func, data)

In [22]:
def fit(epochs, learn):
  for epoch in range(epochs):
    #training loop
    learn.model.train()
    for xb, yb in learn.data.train_dl:
      loss = learn.loss_func(learn.model(xb), yb)
      loss.backward()
      learn.opt.step()
      learn.opt.zero_grad()
    #validation
    learn.model.eval
    with torch.no_grad():
      tot_loss, tot_acc = 0., 0.
      for xb, yb in learn.data.valid_dl:
        pred = learn.model(xb)
        tot_loss += learn.loss_func(pred, yb)
        tot_acc += accuracy(pred, yb)

      num_vars = len(learn.data.valid_dl)
      print(epoch, tot_loss/num_vars, tot_acc/num_vars)

    return tot_loss/num_vars, tot_acc/num_vars 

In [23]:
loss, acc = fit(1, learn)

0 tensor(0.1968) tensor(0.9387)


In [ ]:
#Now that we have out fit, we can add callbacks to make it more flexible for an array of 'ifs' without adding 100000 if-else statements
def one_batch(xb,yb,cb):
  if not cb.begin_batch(xb,yb): return
  loss = cb.learn.loss_func(cb.learn.model(xb),yb)
  if not cb.after_loss(loss) return
  loss.backward()
  if cb.after_backward(): cb.learn.opt.step()
  if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb):
  for xb,yb in dl:
    one_batch(xb,yb,cb)
    if cb.do_stop(): return 